### MARKDOWN


In [26]:
# 테이블 설정 로드
from omegaconf import OmegaConf
config = OmegaConf.load("config/credentials.yml")

In [2]:
#
from transformers import pipeline
classifier = pipeline('zero-shot-classification', model='facebook/bart-large-mnli')

labels = ['spam', 'not spam']
hypothesis_template = 'This email is {}.'
email = 'click this link for sale!'
results = classifier(
        email, labels, hypothesis_template=hypothesis_template
    )

In [9]:
results['labels'][0]

'not spam'

In [26]:
from openai import OpenAI
import pandas as pd

# OpenAI API 키 설정
api_key = config['openai_key']
client = OpenAI(api_key=api_key)


# 데이터
data = {
    "PRODUCT ID": ["B001E4KFG0", "B00813GRG4"],
    "USER ID": ["A3SGXH7AUHU8GW", "A1D87F6ZCVE5NK"],
    "SCORE": [5, 1],
    "SUMMARY": ["Good Quality Dog Food", "Not as Advertised"],
    "TEXT": [
        "I have bought several of the Vitality canned...",
        "Product arrived labeled as Jumbo Salted Peanut..."
    ]
}

# DataFrame 생성
df = pd.DataFrame(data)
df["combined"] = (
    "Title: " + df.SUMMARY.str.strip() + "; Content: " + df.TEXT.str.strip()
)


def get_embedding(text, model="text-embedding-3-small"):
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model).data[0].embedding


df['ada_embedding'] = df.combined.apply(lambda x: get_embedding(x, model='text-embedding-3-small'))


# 여러 텍스트에 대한 임베딩 얻기
embeddings = [get_embedding(text) for text in texts]

# for text, embedding in zip(texts, embeddings):
#     print(f"Text: {text}")
#     print(f"Embedding: {embedding}")


### 사전 학습모델로 임베딩하기

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer(
    'sentence-transformers/multi-qa-mpnet-base-cos-v1'
)

docs =[
    "Around 9 million people live in London",
    "London is known for its financial district"
]

doc_emb = model.encode(
    docs,
    batch_size=32,
    show_progress_bar=True
)

In [53]:
import PyPDF2
from tqdm import tqdm

with open('./data/pds2.pdf', 'rb') as file:
    reader = PyPDF2.PdfReader(file)

    principles_of_ds = ''

    for page in tqdm(reader.pages):
        text = page.extract_text()

        principles_of_ds += '\n\n' + text[text.find(' ]')+2:]

principles_of_ds = principles_of_ds.strip()

100%|██████████| 428/428 [13:01<00:00,  1.83s/it]


### 중첩을 포함하는 또는 포함하지 않는 교과서 분할하기

In [54]:
def overlapping_chunks(text, max_tokens = 500, overlapping_factor = 5):
    '''
    max_tokens: 각 조강에 들어갈 최대 토큰 수
    overlapping_factor: 각 조각이 시작할 때 이전 청크와 중첩되는 문장의 숫자
    '''

    # Split the text using punctuation
    sentences = re.split(r' *[\.\?!][\'"\)\]]* *', text)

    #BERT로 token화 -> 사전 학습된 언어 모델의 어휘(vocabulary)에 매핑,  각 숫자는 텍스트의 특정 단어 또는 하위 단어(subword)에 해당
    # 각 문장의 토큰 수 얻기
    n_tokens = [len(tokenizer.encode(" " + sentence)) for sentence in sentences]

    chunks, tokens_so_far, chunk = [], 0, []

    # 튜플로 결합된 문장과 토큰을 반복해서 처리하기
    for sentence, token in zip(sentences, n_tokens):
        if tokens_so_far + token > max_tokens:
            #max_token수를 넘기면 chunk들을 paste 후 chunks 목록에 추가하기
            #chunk 리셋
            chunks.append(". ".join(chunk) + ".")
            if overlapping_factor > 0:
                chunk = chunk[-overlapping_factor:]
                tokens_so_far = sum([len(tokenizer.encode(c)) for c in chunk])
            else:
                chunk = []
                tokens_so_far = 0
        
        if token > max_tokens:
            continue

        chunk.append(sentence)
        tokens_so_far += token + 1

    if chunk:
        chunks.append(". ".join(chunk) + ".")
    
    return chunks


In [56]:
split = overlapping_chunks(principles_of_ds, overlapping_factor=0)
avg_length =sum([len(tokenizer.encode(t)) for t in split]) / len(split)

In [57]:
print(f'non-overlapping chunking approach has {len(split)} documents with average length {avg_length:.1f} tokens')

non-overlapping chunking approach has 299 documents with average length 443.0 tokens


In [59]:
split = overlapping_chunks(principles_of_ds, overlapping_factor=5)
avg_length =sum([len(tokenizer.encode(t)) for t in split]) / len(split)

In [60]:
print(f'overlapping chunking approach has {len(split)} documents with average length {avg_length:.1f} tokens')

overlapping chunking approach has 420 documents with average length 451.4 tokens


### 감정 훈련 데이터를 위한 JSONL 파일 생성

In [1]:
from datasets import load_dataset
import pandas as pd

# 아마존 리뷰 다중 언어 데이터셋을 로드
# dataset = load_dataset("amazon_reviews_multi", "all_languages")
dataset = load_dataset('amazon_polarity')

#데이터셋의 'train' 서브셋을 pandas DataFrame
training_df = pd.DataFrame(dataset['train'])

In [14]:
training_df_s = training_df.head()

In [15]:
training_df_s

,label,title,content
0,1,Stuning even for the non-gamer,This sound track was beautiful! It paints the ...
1,1,The best soundtrack ever to anything.,I'm reading a lot of reviews saying that this ...
2,1,Amazing!,This soundtrack is my favorite music of all ti...
3,1,Excellent Soundtrack,I truly like this soundtrack and I enjoy video...
4,1,"Remember, Pull Your Jaw Off The Floor After He...","If you've played the game, you know how divine..."


In [16]:
def prepare_df_for_openai(df):
    
    # 'reviw_title' 과 'review_body' 열을 결합하고,
    # 끝에 맞춤형 접미사 '\n\n###\n\n'를 추가하여 'prompt'열을 생성
    df['prompt'] = df['title'] + '\n\n' + df['content'] + '\n\n###\n\n'

    # 'stars' 값 앞에 공백을 추가함으로써 새로운 'completion' 열을 생성
    df['completion'] = ' ' + df['label'].astype(str)

    english_df = df

    # 'prompt' 열을 기준으로 중복 행을 제거
    english_df.drop_duplicates(subset=['prompt'], inplace=True)

    return english_df[['prompt', 'completion']].sample(len(english_df))
    
english_training_df = prepare_df_for_openai(training_df_s)

english_training_df.to_json("amazon-english-full-train-sentiment.jsonl", orient='records', lines=True)


C:\Users\User\AppData\Local\Temp\ipykernel_7616\1335550975.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['prompt'] = df['title'] + '\n\n' + df['content'] + '\n\n###\n\n'
C:\Users\User\AppData\Local\Temp\ipykernel_7616\1335550975.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['completion'] = ' ' + df['label'].astype(str)
C:\Users\User\AppData\Local\Temp\ipykernel_7616\1335550975.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats

### OpenAI API로부터 토큰 확률 얻기

In [21]:
test_df = pd.DataFrame(dataset['test'])

In [22]:
english_test_df = prepare_df_for_openai(test_df)

In [80]:
import math
from openai import OpenAI
import pandas as pd

# OpenAI API 키 설정
api_key = config['openai_key']
client = OpenAI(api_key=api_key)



# 샘플 프롬프트 가져오기
prompt = english_test_df['prompt'].sample(1).iloc[0]

# 파인튜닝된 모델을 사용하여 완성본 생성하기
response = client.chat.completions.create(
    # model='ada:ft-personal-2023-03-31-05-30-46',
    model= 'gpt-3.5-turbo',
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ],
    max_tokens=1,
    temperature=0,
    logprobs=True
)


# 결과 처리
probs = []
logprobs = response.choices[0].logprobs.content[0].top_logprobs
    
for logprob in logprobs:
    _probs = {}
    for key, value in logprob.items():
        _probs[key] = math.exp(value)
    probs.append(_probs)




